# <font color='#1E3E62'>_**Customer Segmentation Model**_</font> 

## <font color='#FF6500'>_**Transactions - Categories**_</font> 

## <font color= '#1E3E62'>**_Recommend the best merchants for each user as targeted offers_**</font> 

![title](CS.jpg)

### 📌 <font color='#1E3E62'>_**To begin, let's identify the key points & outline the questions that will guide 'our analysis', helping us define the 'business questions' and 'understand the dataset's structure'.:**_</font> 

<font color = '#FF6500' font size="4">**_(I) Business Questions:_**</font>

<font size="3">**_1. How can we optimize customer retention by analyzing transaction behavior?_**</font>

<font size="3">**_2. Can we predict customer loyalty based on transaction ranks and value?_**</font>

<font size="3">**_3. How do customer transactions vary across different merchant categories?_**</font>

<font color = '#FF6500' font size="4">**_(II) Business Objectives:_**</font>

<font size="3">**_1. Customer Retention:_**</font>

>- Identify key factors that affect customer loyalty and retention.

<font size="3">**_2. Customer Segmentation:_**</font>

>- Group customers into segments based on transaction behavior to target marketing efforts.

<font size="3">**_3. Merchant Insights:_**</font>

>- Understand which merchant categories drive the highest transaction values or redeemed points.

<font color = '#FF6500' font size="4">**_(III) Key Questions:_**</font>

<font size="3">**_1. What are the top categories or merchants contributing the most to transaction value?_**</font>

<font size="3">**_2. Which customers are most loyal; based on their transaction rank and days since last transaction?_**</font>

<font size="3">**_3. Are there patterns in the redeemed points across different merchants or categories?_**</font>

<font color = '#FF6500' font size="4">**_(VI) Dataset Structure Understanding:_**</font>

<font color = '#1E3E62' font size="3">_**The Transactions dataset provides individual transaction records for customers, including details about the merchant, the points redeemed, and the transaction value.**_</font>

>- **CustomerID:** A unique identifier for each customer. **(This will be useful for grouping and segmenting the customers based on their transactional history.)**
>- **MerchantName:** Name of the merchant where the transaction took place. **(We can map this with the Categories dataset to understand the industry/sector of the merchant.)**
>- **TransactionRank:** An integer representing the rank or priority of the transaction. **(This can give insights into customer behavior based on transaction frequency.)**
>- **TransactionRedeemedPoints:** Points that were redeemed during the transaction. **(This is useful for understanding customer engagement with reward programs.)**
>- **TransactionValue:** The monetary value of the transaction. **(Higher values might indicate high-value customers.)**
>- **TransactionFrom(days):** The number of days since the first transaction of the customer. **(This helps to analyze customer longevity and consistency.)**
>- **CustomerLastTransactionFrom(days):** The number of days since the customer’s last transaction. **(This is useful for identifying potential churn (if the customer hasn’t transacted in a while).)**

<font color = '#1E3E62' font size="3">_**The Categories dataset maps each merchant to a category (e.g., fashion, grocery, F&B), which can help in further categorizing the transaction data for deeper insights.**_</font>

<font color = '#FF6500' font size="4">**_(V) Business Impact:_**</font>

<font size="3">_The analysis will have a significant impact on new restaurant planning by helping:_</font>

>- **Targeted Marketing:** By segmenting customers based on their transaction frequency, value, and recency, we can target high-value or high-potential customers with personalized offers.
>- **Customer Loyalty Programs:** Understanding how often and how many points customers redeem can help refine loyalty programs and reward high-frequency customers.
>- **Merchant Strategy:** Knowing which merchant categories are generating the most value allows businesses to build strategic partnerships and promotions with those categories.


## 📌 <font color='#FF6500'>*Target Tasks:*</font> 
<font size="4">**_1- Data Processing_**</font>
<br>
<font size="4">**_2- EDA (Exploratory of Data Analysis)_**</font>
<br>
<font size="4">**_3- Data Preprocessing_**</font>
<br>
<font size="4">**_4- Model Data_**</font>
<br>
<font size="4">**_5- Evaluation and Tuning_**</font>
<br>
<font size="4">**_6- Deployment_**</font>

# 📌 <font color='#1E3E62'>*_1- Data Processing:_*</font> 

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import hvplot.pandas
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
# Load the Transactions & Categories datasets
transactions_df = pd.read_csv('Transactions.csv')
categories_df = pd.read_csv('Categories.csv')

In [ ]:
# Read the Transactions dataset
transactions_df.head(-10)

In [ ]:
# Read the Categories dataset
categories_df.head(60)

In [ ]:
# Get the info of Transactions dataset
transactions_df.info()

In [ ]:
# Get the info of Categories dataset
categories_df.info()

In [ ]:
# Check for missing data in Transactions Dataset
transactions_df.isnull().sum()

In [ ]:
# Check for missing data in Categories Dataset
transactions_df.isnull().sum()

In [ ]:
# Check for duplicate rows in Transactions Dataset
transactions_df.duplicated().sum()

In [ ]:
# Check for duplicate rows in Categories Dataset
categories_df.duplicated().sum()

In [ ]:
# Check if all merchants in Transactions Dataset have categories in Categories Dataset
missing_categories = transactions_df[~transactions_df['MerchantName'].isin(categories_df['MerchantName'])]

In [ ]:
missing_categories

In [ ]:
transactions_df.describe()

In [ ]:
transactions_df.columns

In [ ]:
transactions_df["MerchantName"].value_counts()

In [ ]:
sorted(transactions_df["MerchantName"].unique())

In [ ]:
transactions_df["TransactionRank"].value_counts()

In [ ]:
px.histogram(data_frame=transactions_df , x = 'TransactionRank')

In [ ]:
transactions_df["TransactionRedeemedPoints"].value_counts()

In [ ]:
px.histogram(data_frame=transactions_df , x = 'TransactionRedeemedPoints')

In [ ]:
transactions_df["TransactionRedeemedPoints"].describe()

In [ ]:
transactions_df["TransactionValue"].value_counts()

In [ ]:
px.histogram(data_frame=transactions_df , x = 'TransactionValue')

In [ ]:
transactions_df["TransactionValue"].describe()

In [ ]:
# Remove negative or zero values for TransactionValue
transactions_df.drop(transactions_df[transactions_df['TransactionValue'] <= 0].index, inplace=True)

In [ ]:
# Confirm the removal by checking for any remaining negative or zero values
transactions_df[transactions_df['TransactionValue'] <= 0]

# Reset Index
transactions_df.reset_index(drop = True , inplace = True)

In [ ]:
transactions_df.head(-10)

In [ ]:
# Calculate IQR for TransactionValue and TransactionRedeemedPoints
Q1 = transactions_df[['TransactionValue', 'TransactionRedeemedPoints']].quantile(0.25)
Q3 = transactions_df[['TransactionValue', 'TransactionRedeemedPoints']].quantile(0.75)
IQR = Q3 - Q1

# Identify outliers for TransactionValue and TransactionRedeemedPoints using the IQR method
transaction_outliers = (transactions_df < (Q1 - 1.5 * IQR)) | (transactions_df > (Q3 + 1.5 * IQR))
transaction_outliers_df = transactions_df[transaction_outliers.any(axis=1)]

# Review the first few rows of IQR outliers
transaction_outliers_df.head() 

In [ ]:
# IQR Method: Count the number of outliers in TransactionValue and TransactionRedeemedPoints
transaction_outliers_count = transaction_outliers_df.shape[0]

print(f"Number of outliers identified using the IQR method: {transaction_outliers_count}")

In [ ]:
# Step 2: Identify outliers using Z-scores
z_scores = np.abs(stats.zscore(transactions_df[['TransactionValue', 'TransactionRedeemedPoints']]))

# Define a threshold for Z-scores (commonly 3 is used to identify outliers)
z_score_threshold = 3
outliers_zscore = (z_scores > z_score_threshold)
outliers_zscore_df = transactions_df[outliers_zscore.any(axis=1)]

# Review the first few rows of Z-score outliers
outliers_zscore_df.head()  

In [ ]:
# Z-Score Method: Count the number of outliers in TransactionValue and TransactionRedeemedPoints
outliers_zscore_count = outliers_zscore_df.shape[0]

print(f"Number of outliers identified using the Z-Score method: {outliers_zscore_count}")

In [ ]:
# Combine the IQR and Z-Score outliers into one dataframe and remove duplicates
combined_outliers_df = pd.concat([transaction_outliers_df, outliers_zscore_df]).drop_duplicates()

# Save the combined outliers dataset to a CSV file
combined_outliers_df.to_csv("combined_outliers.csv", index=False)

In [ ]:
px.histogram(data_frame=combined_outliers_df , x = 'TransactionValue')

In [ ]:
px.histogram(data_frame=combined_outliers_df , x = 'TransactionRedeemedPoints')

>- <font color = '#1E3E62' font size="5">_**Since the outliers are legitimate transactions that reflect customer behavior (e.g., a customer makes an unusually large purchase or redeems a large number of points).**_</font>
>- <font color = '#FF6500' font size="5">_**I decided keeping them, as they could provide insights into high-value customers or infrequent but significant transactions.**_</font>

# 📌 <font color='#1E3E62'>*_2- EDA (Exploratory of Data Analysis):_*</font> 

## <font color='#FF6500'>*Univariate:*</font> 

In [ ]:
# Insight 1: Distribution of transactions across different MerchantNames (top 10 merchants)
merchant_count = transactions_df['MerchantName'].value_counts().nlargest(10).reset_index()
merchant_count.columns = ['MerchantName', 'TransactionCount']
fig1 = px.bar(merchant_count, x='MerchantName', y='TransactionCount', title='Top 10 Merchants by Transaction Count')
fig1.show()

# Insight 2: Count of transactions based on TransactionRank
transaction_rank_count = transactions_df['TransactionRank'].value_counts().reset_index()
transaction_rank_count.columns = ['TransactionRank', 'Count']
fig2 = px.bar(transaction_rank_count, x='TransactionRank', y='Count', title='Count of Transactions by TransactionRank')
fig2.show()

# Insight 3: Number of transactions for each CustomerID (top 10 customers)
customer_count = transactions_df['CustomerID'].value_counts().nlargest(10).reset_index()
customer_count.columns = ['CustomerID', 'TransactionCount']
fig3 = px.bar(customer_count, x='CustomerID', y='TransactionCount', title='Top 10 Customers by Transaction Count')
fig3.show()

# Insight 4: Frequency of customers based on TransactionRedeemedPoints brackets
transactions_df['RedeemedPointsBracket'] = pd.cut(transactions_df['TransactionRedeemedPoints'], bins=[0, 1000, 5000, 10000, 50000], labels=['0-1000', '1000-5000', '5000-10000', '10000+'])
redeemed_points_count = transactions_df['RedeemedPointsBracket'].value_counts().reset_index()
redeemed_points_count.columns = ['RedeemedPointsBracket', 'Count']
fig4 = px.bar(redeemed_points_count, x='RedeemedPointsBracket', y='Count', title='Frequency of Customers Based on Redeemed Points Brackets')
fig4.show()

# Insight 5: Count of transactions based on Merchant Category (using the Categories.csv dataset)
transactions_with_category = pd.merge(transactions_df, categories_df, how='left', left_on='MerchantName', right_on='MerchantName')
merchant_category_count = transactions_with_category['Category'].value_counts().reset_index()
merchant_category_count.columns = ['Category', 'TransactionCount']
fig5 = px.bar(merchant_category_count, x='Category', y='TransactionCount', title='Count of transactions based on Merchant Category')
fig5.show()


In [ ]:
# Insight 6: Scatter plot of CustomerID vs. TransactionValue
fig6 = px.scatter(transactions_df, x='CustomerID', y='TransactionValue', title='CustomerID vs. Transaction Value')
fig6.show()

# Insight 7: Scatter plot of CustomerLastTransactionFrom(days)
fig7 = px.scatter(transactions_df, x='CustomerID', y='CustomerLastTransactionFrom(days)', title='CustomerID vs. CustomerLastTransactionFrom(days)')
fig7.show()

# Insight 8: Scatter plot of TransactionValue
fig8 = px.scatter(transactions_df, y='TransactionValue', title='Transaction Value Distribution')
fig8.show()

# Insight 9: Scatter plot of TransactionRedeemedPoints across customers
fig9 = px.scatter(transactions_df, x='CustomerID', y='TransactionRedeemedPoints', title='CustomerID vs. Transaction Redeemed Points')
fig9.show()

# Insight 10: Scatter plot of TransactionRank
fig10 = px.scatter(transactions_df, y='TransactionRank', title='Transaction Rank Distribution')
fig10.show()

In [ ]:
# Insight 11: Pie chart of the proportion of transactions by MerchantName (top 10 merchants)
merchant_transaction_count = transactions_df['MerchantName'].value_counts().nlargest(10)
fig11 = px.pie(merchant_transaction_count, values=merchant_transaction_count, names=merchant_transaction_count.index, title='Top 10 Merchants by Transaction Count')
fig11.show()

# Insight 12: Pie chart of the total number of TransactionRanks
transaction_rank_count = transactions_df['TransactionRank'].value_counts()
fig12 = px.pie(transaction_rank_count, values=transaction_rank_count, names=transaction_rank_count.index, title='Distribution of TransactionRank')
fig12.show()

# Insight 13: Pie chart of total transaction count by Merchant Category
merchant_category_count = transactions_with_category['Category'].value_counts()
fig13 = px.pie(merchant_category_count, values=merchant_category_count, names=merchant_category_count.index, title='Transaction Count by Merchant Category')
fig13.show()

# Insight 14: Pie chart of the percentage of customers based on TransactionRedeemedPoints ranges
redeemed_points_bracket = transactions_df['RedeemedPointsBracket'].value_counts()
fig14 = px.pie(redeemed_points_bracket, values=redeemed_points_bracket, names=redeemed_points_bracket.index, title='Redeemed Points Distribution')
fig14.show()

# Insight 15: Pie chart showing TransactionValue contribution by the top 5 merchants
top_5_merchants = transactions_df.groupby('MerchantName')['TransactionValue'].sum().nlargest(5)
fig15 = px.pie(top_5_merchants, values=top_5_merchants, names=top_5_merchants.index, title='Top 5 Merchants by Transaction Value')
fig15.show()

In [ ]:
# Insight 16: Histogram of TransactionValue
fig16 = px.histogram(transactions_df, x='TransactionValue', nbins=20, title='Transaction Value Distribution')
fig16.show()

# Insight 17: Histogram of TransactionRedeemedPoints
fig17 = px.histogram(transactions_df, x='TransactionRedeemedPoints', nbins=20, title='Redeemed Points Distribution')
fig17.show()

# Insight 18: Histogram of TransactionFrom(days)
fig18 = px.histogram(transactions_df, x='TransactionFrom(days)', nbins=20, title='Transaction From (days) Distribution')
fig18.show()

# Insight 19: Histogram of CustomerLastTransactionFrom(days)
fig19 = px.histogram(transactions_df, x='CustomerLastTransactionFrom(days)', nbins=20, title='Customer Last Transaction From (days) Distribution')
fig19.show()

# Insight 20: Histogram of TransactionRank
fig20 = px.histogram(transactions_df, x='TransactionRank', nbins=10, title='Transaction Rank Distribution')
fig20.show()

In [ ]:
# Insight 21: KDE plot of TransactionValue
fig21 = px.density_contour(transactions_df, x='TransactionValue', title='KDE Plot of Transaction Value')
fig21.show()

# Insight 22: KDE plot of TransactionRedeemedPoints
fig22 = px.density_contour(transactions_df, x='TransactionRedeemedPoints', title='KDE Plot of Redeemed Points')
fig22.show()

# Insight 23: KDE plot of CustomerLastTransactionFrom(days)
fig23 = px.density_contour(transactions_df, x='CustomerLastTransactionFrom(days)', title='KDE Plot of Customer Last Transaction From (days)')
fig23.show()

# Insight 24: KDE plot of TransactionFrom(days)
fig24 = px.density_contour(transactions_df, x='TransactionFrom(days)', title='KDE Plot of Transaction From (days)')
fig24.show()

# Insight 25: KDE plot of TransactionValue for the top 10 merchants
top_10_merchants = transactions_df[transactions_df['MerchantName'].isin(transactions_df['MerchantName'].value_counts().nlargest(10).index)]
fig25 = px.density_contour(top_10_merchants, x='TransactionValue', color='MerchantName', title='KDE Plot of Transaction Value for Top 10 Merchants')
fig25.show()


## <font color='#FF6500'>*Bivariate:*</font> 

In [ ]:
# 1. Bar Plots 

# Insight 26: Bar plot of TransactionValue by MerchantName
fig26 = px.bar(transactions_df.groupby('MerchantName')['TransactionValue'].sum().reset_index(), 
               x='MerchantName', y='TransactionValue', title='Transaction Value by Merchant Name')
fig26.show()

# Insight 27: Bar plot of TransactionRedeemedPoints by MerchantName
fig27 = px.bar(transactions_df.groupby('MerchantName')['TransactionRedeemedPoints'].sum().reset_index(), 
               x='MerchantName', y='TransactionRedeemedPoints', title='Transaction Redeemed Points by Merchant Name')
fig27.show()

# Insight 28: Bar plot of TransactionValue by TransactionRank
fig28 = px.bar(transactions_df.groupby('TransactionRank')['TransactionValue'].sum().reset_index(), 
               x='TransactionRank', y='TransactionValue', title='Transaction Value by Transaction Rank')
fig28.show()

# Insight 29: Bar plot of TransactionValue by CustomerID
fig29 = px.bar(transactions_df.groupby('CustomerID')['TransactionValue'].sum().nlargest(10).reset_index(), 
               x='CustomerID', y='TransactionValue', title='Top 10 Customers by Transaction Value')
fig29.show()

# Insight 30: Bar plot of TransactionValue by CustomerLastTransactionFrom(days)
fig30 = px.bar(transactions_df.groupby('CustomerLastTransactionFrom(days)')['TransactionValue'].sum().reset_index(), 
               x='CustomerLastTransactionFrom(days)', y='TransactionValue', title='Transaction Value by Days Since Last Transaction')
fig30.show()

In [ ]:
# 2. Scatter Plots 

# Insight 31: Scatter plot of TransactionValue vs TransactionRedeemedPoints
fig31 = px.scatter(transactions_df, x='TransactionValue', y='TransactionRedeemedPoints', 
                   title='Transaction Value vs Redeemed Points')
fig31.show()

# Insight 32: Scatter plot of TransactionValue vs TransactionRank
fig32 = px.scatter(transactions_df, x='TransactionRank', y='TransactionValue', 
                   title='Transaction Value vs Transaction Rank')
fig32.show()

# Insight 33: Scatter plot of TransactionFrom(days) vs TransactionValue
fig33 = px.scatter(transactions_df, x='TransactionFrom(days)', y='TransactionValue', 
                   title='Transaction From (days) vs Transaction Value')
fig33.show()

# Insight 34: Scatter plot of CustomerLastTransactionFrom(days) vs TransactionValue
fig34 = px.scatter(transactions_df, x='CustomerLastTransactionFrom(days)', y='TransactionValue', 
                   title='Customer Last Transaction From (days) vs Transaction Value')
fig34.show()

# Insight 35: Scatter plot of TransactionRedeemedPoints vs CustomerLastTransactionFrom(days)
fig35 = px.scatter(transactions_df, x='CustomerLastTransactionFrom(days)', y='TransactionRedeemedPoints', 
                   title='Redeemed Points vs Days Since Last Transaction')
fig35.show()


In [ ]:
# 3. Pie Charts

# Insight 36: Pie chart of TransactionValue proportion by Merchant Category
merchant_category_value = transactions_with_category.groupby('Category')['TransactionValue'].sum().reset_index()
fig36 = px.pie(merchant_category_value, values='TransactionValue', names='Category', 
               title='Transaction Value Proportion by Merchant Category')
fig36.show()

# Insight 37: Pie chart of TransactionValue vs TransactionRank
transaction_value_rank = transactions_df.groupby('TransactionRank')['TransactionValue'].sum().reset_index()
fig37 = px.pie(transaction_value_rank, values='TransactionValue', names='TransactionRank', 
               title='Transaction Value by Transaction Rank')
fig37.show()

# Insight 38: Pie chart of TransactionRedeemedPoints proportion by MerchantName
merchant_redeemed_points = transactions_df.groupby('MerchantName')['TransactionRedeemedPoints'].sum().nlargest(10).reset_index()
fig38 = px.pie(merchant_redeemed_points, values='TransactionRedeemedPoints', names='MerchantName', 
               title='Top 10 Merchants by Redeemed Points')
fig38.show()

# Insight 39: Pie chart of TransactionValue across different ranges of CustomerLastTransactionFrom(days)
transactions_df['TransactionValueBracket'] = pd.cut(transactions_df['CustomerLastTransactionFrom(days)'], 
                                                    bins=[0, 100, 200, 300, 400], labels=['0-100', '100-200', '200-300', '300-400'])
fig39 = px.pie(transactions_df.groupby('TransactionValueBracket')['TransactionValue'].sum().reset_index(), 
               values='TransactionValue', names='TransactionValueBracket', 
               title='Transaction Value by Days Since Last Transaction Bracket')
fig39.show()

# Insight 40: Pie chart of the number of TransactionRanks by Merchant Category
rank_category_count = transactions_with_category.groupby('Category')['TransactionRank'].count().reset_index()
fig40 = px.pie(rank_category_count, values='TransactionRank', names='Category', 
               title='Transaction Rank by Merchant Category')
fig40.show()


In [ ]:
# 4. Histograms

# Insight 41: Histogram of TransactionValue by TransactionRedeemedPoints range
fig41 = px.histogram(transactions_df, x='TransactionValue', color='RedeemedPointsBracket', 
                     title='Transaction Value by Redeemed Points Range')
fig41.show()

# Insight 42: Histogram of TransactionValue across different Merchant Categories
fig42 = px.histogram(transactions_with_category, x='TransactionValue', color='Category', 
                     title='Transaction Value by Merchant Category')
fig42.show()

# Insight 43: Histogram of TransactionValue across different ranges of CustomerLastTransactionFrom(days)
fig43 = px.histogram(transactions_df, x='TransactionValue', color='TransactionValueBracket', 
                     title='Transaction Value by Days Since Last Transaction Bracket')
fig43.show()

# Insight 44: Histogram of TransactionValue by TransactionRank
fig44 = px.histogram(transactions_df, x='TransactionValue', color='TransactionRank', 
                     title='Transaction Value by Transaction Rank')
fig44.show()

# Insight 45: Histogram of TransactionRedeemedPoints by TransactionRank
fig45 = px.histogram(transactions_df, x='TransactionRedeemedPoints', color='TransactionRank', 
                     title='Redeemed Points by Transaction Rank')
fig45.show()

In [ ]:
# Calculate Spearman correlation matrix
spearman_corr = transactions_df.corr(method='spearman')

In [ ]:
# Plot Heatmap for Spearman Correlation
plt.figure(figsize=(10, 8))
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Heatmap of Spearman Correlation Matrix')
plt.show()

In [ ]:
# Display the Spearman correlation matrix
print(spearman_corr)

<font color = '#FF6500' font size="4">**_1- TransactionRank:_**</font>

<font size="3">_**a) Correlation with TransactionValue & TransactionRedeemedPoints: −0.178:**_</font>

>- **Both correlations are weak and negative:** This is indicating that higher **TransactionRank** tends to be associated with slightly **lower TransactionValue and TransactionRedeemedPoints**. However, since the correlation is not strong, rank only slightly impacts how much customers spend or how many points they redeem.

<font size="3">_**b) Correlation with TransactionFrom(days): −0.152:**_</font>

>- **There is a weak negative correlation:** This is suggesting that transactions made further in the past tend to have slightly lower ranks.


<font size="3">_**c) Correlation with CustomerLastTransactionFrom(days): −0.214:**_</font>

>- **This shows a weak negative correlationn:** Customers who have not transacted recently tend to have lower-ranked transactions, though the correlation is still quite weak.

<font color = '#FF6500' font size="4">**_2. TransactionRedeemedPoints & TransactionValue:_**</font>

<font size="3">_**Correlation: 1.000**_</font>

>- **These two features have a perfect positive correlation:** This means that **TransactionRedeemedPoints and TransactionValue are identical or directly proportional in this dataset**. In practical terms, the amount spent in a transaction is exactly matched by the points redeemed, or at least they track each other perfectly.

<font color = '#FF6500' font size="4">**_3. TransactionFrom(days):_**</font>

<font size="3">_**Correlation with CustomerLastTransactionFrom(days): 0.694**_</font>

>- **There is a strong positive correlation:** This suggests that customers who made their first transaction a long time ago also tend to have their most recent transaction further in the past, which is expected. **The correlation is not perfect but shows a strong tendency.**

<font color = '#FF6500' font size="4">**_4. CustomerLastTransactionFrom(days):_**</font>

<font size="3">_**Correlation with Other Variables:**_</font>

>- **The correlation with all other variables except TransactionFrom(days) is very weak (below 0.2):** This implies that how long it has been since the customer’s last transaction does not strongly relate to any other feature except for how long ago the customer made their first transaction.

<font color = '#FF6500' font size="4">**Summary of Insights:_**</font>

<font size="3">_**TransactionRedeemedPoints and TransactionValue are perfectly correlated**_</font>
>- _This suggests that for each transaction, the points redeemed are directly proportional to the transaction value._

<font size="3">_**TransactionRank has a weak negative relationship with TransactionValue and TransactionRedeemedPoints**_</font>
>- _This is meaning higher transaction ranks slightly correlate with lower spending and points redeemed, but the effect is not very strong._

<font size="3">_**TransactionFrom(days) and CustomerLastTransactionFrom(days) are strongly positively correlated, which makes intuitive sense:**_</font>
>- _Customers who have been with the business longer (earlier first transactions) also tend to have had more recent transactions further in the past._

<font size="3">_**Most other relationships are quite weak**_</font>
> - _This is meaning the variables are largely independent of each other, or their relationships are non-linear._

# 📌 <font color='#1E3E62'>*_3- Data Preprocessing:_*</font> 

<font color = '#1E3E62' font size="4">**_Steps for RFM Clustering:_**</font>

<font color = '#FF6500' font size="4">**_1. Calculate RFM Scores:_**</font>

>- **Recency:** The number of days since the customer's last transaction.
>- **Frequency:** The total number of transactions for each customer.
>- **Monetary:** The total value of transactions for each customer.

<font color = '#FF6500' font size="4">**_2. Scale the Features:_**</font>
>- Apply standard scaling to the RFM values to normalize them for clustering.

<font color = '#FF6500' font size="4">**_3. Apply KMeans Clustering:_**</font>
>- Use KMeans to segment customers based on their RFM scores.

<>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

<font color = '#FF6500' font size="4">**_After completing the above steps:_**</font>

>- **Recency:** Lower values indicate that customers have transacted recently.
>- **Frequency:** Higher values indicate more frequent transactions.
>- **Monetary:** Higher values indicate that customers spend more.

<font color = '#FF6500' font size="4">_The resulting clusters will show you which group of customers are **frequent, high-value, or lapsed** based on their RFM scores._</font>

In [ ]:
# Calculate Recency, Frequency, and Monetary value for each customer
rfm_df = transactions_df.groupby('CustomerID').agg({
    'CustomerLastTransactionFrom(days)': 'min',  # Recency
    'TransactionRank': 'count',                 # Frequency
    'TransactionValue': 'sum'                   # Monetary value
}).reset_index()

# Rename columns for clarity
rfm_df.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

In [ ]:
rfm_df.head(100)

In [ ]:
# Step 2: Standardize the RFM values
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_df[['Recency', 'Frequency', 'Monetary']])

In [ ]:
# Step 3: Apply KMeans Clustering (You can adjust the number of clusters)
kmeans = KMeans(n_clusters=4, random_state=42)
rfm_df['RFM_Cluster'] = kmeans.fit_predict(rfm_scaled)

In [ ]:
# Step 4: Analyze the resulting clusters
cluster_analysis = rfm_df.groupby('RFM_Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'CustomerID': 'count'  # Number of customers in each cluster
}).reset_index()

print(cluster_analysis)

<font color = '#1E3E62' font size="4">_**Cluster Interpretation:**_</font>

<font color = '#FF6500' font size="4">_**Cluster 0:**_</font>

>- **Recency (62 days):** These customers have made recent purchases but are not the most recent.
>- **Frequency (1.2 transactions):** They make infrequent purchases.
>- **Monetary (372.93):** They spend a moderate amount when they purchase.
>- **Interpretation:** <font color = '#0F6292'>_**Hibernating**_</font> ; <font color = '#FF0075'>_**These customers are at risk of becoming inactive, and they spend a moderate amount. You may want to re-engage them with personalized offers or discounts.**_</font>

<font color = '#FF6500' font size="4">_**Cluster 1:**_</font>

>- **Recency (268 days):** These customers have not purchased in a long time.
>- **Frequency (1.07 transactions):** They rarely make purchases.
>- **Monetary (346.45):** They spend a relatively low amount.
>- **Interpretation:** <font color = '#0F6292'>_**At Risk**_</font> ; <font color = '#FF0075'>_**These customers haven’t purchased in a long time and spend little. You should focus on reactivating them with targeted promotions or win-back campaigns.**_</font>

<font color = '#FF6500' font size="4">_**Cluster 2:**_</font>

>- **Recency (43 days):** These customers have made very recent purchases.
>- **Frequency (25 transactions):** They purchase very frequently.
>- **Monetary (144,865.95):** They spend a very high amount.
>- **Interpretation:** <font color = '#0F6292'>_**Champions**_</font> ; <font color = '#FF0075'>_**These are your best customers. They purchase frequently, spend a lot, and have made recent purchases. Focus on retaining them with exclusive offers and loyalty programs.**_</font>

<font color = '#FF6500' font size="4">_**Cluster 3:**_</font>

>- **Recency (80 days):** These customers have made relatively recent purchases.
>- **Frequency (3.93 transactions):** They purchase occasionally.
>- **Monetary (1,557.74):** They spend a moderate amount.
>- **Interpretation:** <font color = '#0F6292'>_**Loyal Customers**_</font> ; <font color = '#FF0075'>_**These customers are loyal and make regular purchases. You can further nurture them to increase their frequency and spending.**_</font>


In [ ]:
# Create a mapping of cluster numbers to their interpretation
cluster_labels = {
    0: 'Hibernating',
    1: 'At Risk',
    2: 'Champions',
    3: 'Loyal Customers'
}

# Add a new column 'Lable' that maps the RFM_Cluster to its interpretation
rfm_df['Label'] = rfm_df['RFM_Cluster'].map(cluster_labels)

# Display the updated dataframe
print(rfm_df[['CustomerID', 'RFM_Cluster', 'Recency', 'Frequency', 'Monetary', 'Label']])

In [ ]:
# Scatter plot of Recency vs Monetary with cluster labels
plt.figure(figsize=(10, 6))
sns.scatterplot(data=rfm_df, x='Recency', y='Monetary', hue='Label', palette='viridis')
plt.title('RFM Clusters: Recency vs Monetary')
plt.show()

In [ ]:
# 3D Scatter Plot using Plotly Express
fig = px.scatter_3d(rfm_df, x='Recency', y='Frequency', z='Monetary', color='Label', title="RFM Clustering in 3D")
fig.show()

In [ ]:
# Bar plot of average Recency, Frequency, and Monetary by Cluster
cluster_agg = rfm_df.groupby('RFM_Cluster').mean()[['Recency', 'Frequency', 'Monetary']].reset_index()

# Plot the aggregated data
cluster_agg.plot(kind='bar', figsize=(10, 6), x='RFM_Cluster')
plt.title('Average RFM Values by Cluster')
plt.ylabel('Average Value')
plt.show()

In [ ]:
sns.pairplot(rfm_df, hue='Label', vars=['Recency', 'Frequency', 'Monetary'], palette='viridis')
plt.title('RFM Cluster Pair Plot')
plt.show()


<font color = '#1E3E62' font size="5">_**Action Plan**_</font>

<font color = '#1E3E62' font size="4">_**1. Champions (Cluster 2)**_</font>

<font color = '#FF6500' font size="4">_**Characteristics:**_</font>:

>- **Recent purchases (Recency: 43 days).**
>- **Frequent buyers (Frequency: 25 transactions).**
>- **High spenders (Monetary: $144,865.95).**

<font color = '#C70039' font size="4">_**Action Plan:**_</font>:

**a) Reward Loyalty:** These are your most valuable customers, so focus on retaining them.
>- Implement a VIP program that offers exclusive benefits (e.g., early access to products, special discounts).
>- Personalized offers or thank-you messages to make them feel appreciated.

**b) Engagement:** Keep them engaged with loyalty points, personalized product recommendations, and regular communication (email, SMS).

**c) Referrals:** Leverage this group for referral programs to bring in new customers.

**d) Feedback:** Ask for feedback to improve your offerings or customer experience.

<font color = '#1E3E62' font size="4">_**2. Loyal Customers (Cluster 3)**_</font>

<font color = '#FF6500' font size="4">_**Characteristics:**_</font>:

>- **Fairly recent purchases (Recency: 80 days).**
>- **Regular buyers (Frequency: 3.93 transactions).**
>- **Moderate spenders (Monetary: $1,557.74).**

<font color = '#C70039' font size="4">_**Action Plan:**_</font>:

**a) Nurture Relationship:** Keep these customers coming back with frequent engagement.
>- Offer loyalty rewards based on their continued patronage (e.g., points, discounts).
>- Provide personalized product recommendations or bundles based on past purchases.

**b) Upsell and Cross-sell:** Introduce them to higher-value products or complementary items to increase their spending.

**c) Exclusive Offers:** Send them occasional exclusive deals or previews to boost their sense of loyalty.

<font color = '#1E3E62' font size="4">_**3. Hibernating (Cluster 0)**_</font>

<font color = '#FF6500' font size="4">_**Characteristics:**_</font>:

>- **Moderately recent purchases (Recency: 62 days).**
>- **Low frequency of purchases (Frequency: 1.2 transactions).**
>- **Moderate spenders (Monetary: $372.93).**

<font color = '#C70039' font size="4">_**Action Plan:**_</font>:

**a) Reactivation Campaigns:** These customers are at risk of becoming inactive.
>- Launch win-back campaigns to re-engage them with incentives like limited-time discounts, free shipping, or other offers.

**b) Highlight New Products:** Show them new arrivals, updates, or popular products to encourage another purchase.

**c) Personalized Offers:** Provide personalized offers based on past purchases to remind them of your brand.

**d) Content Marketing:** Send engaging content, such as newsletters or product tips, to keep them interested in your brand.

<font color = '#1E3E62' font size="4">_**4. At Risk (Cluster 1)**_</font>

<font color = '#FF6500' font size="4">_**Characteristics:**_</font>:

>- **Customers have not purchased in a long time (Recency: 268 days).**
>- **Rare buyers (Frequency: 1.07 transactions).**
>- **Relatively low spenders (Monetary: $346.45).**

<font color = '#C70039' font size="4">_**Action Plan:**_</font>:

**a) Aggressive Re-engagement:** These customers are in danger of churning.
>- Offer significant incentives like heavily discounted items, personalized offers, or free shipping to encourage a comeback.
>- Send "We Miss You" emails or messages with personalized offers tailored to their past purchases.

**b) Surveys:** Ask for feedback to understand why they haven’t purchased recently and address their concerns.

**c) Targeted Ads:** Use retargeting ads to remind them of your brand and promote relevant products.

**d) Win-back Strategies:** Create urgency by offering time-sensitive deals to bring them back.

<font color = '#FF0000' font size="4">_**General Strategies for Each Cluster:**_</font>

**(1) Retention:** The most important action for clusters like Champions and Loyal Customers is to retain them. They are already loyal and high-spending, so your goal is to ensure they continue to engage with your brand by offering consistent value, rewards, and personalized communication.

**(2) Reactivation:** For clusters like Hibernating and At Risk, the goal is to re-engage these customers before they churn completely. This can be done with discounts, personalized offers, and timely communication.

**(3) Segmentation-Based Offers:** Tailor your offers and communication according to each cluster's behavior. For example, offer high-value products to Champions, upselling options to Loyal Customers, and heavy discounts or win-back offers to At Risk and Hibernating customers.

**(4) Personalization:** Across all clusters, personalized communication (such as product recommendations, birthday offers, or special occasions) can help improve customer engagement and drive sales.



# 📌 <font color='#1E3E62'>*_4- Model Data:_*</font> 

<font color = '#FF6500' font size="4">**_Step 1: Clustering Algorithms Overview_**</font>

_We'll try several clustering algorithms that are suitable for customer segmentation:_

>- **1- K-Means:** A classic centroid-based clustering algorithm.
>- **2- DBSCAN (Density-Based Spatial Clustering):** A density-based clustering method that identifies clusters based on dense regions of data points.
>- **3- Agglomerative Hierarchical Clustering:** A hierarchical clustering method that creates clusters by merging or splitting existing clusters based on distance.
>- **4- Gaussian Mixture Model (GMM):** A probabilistic model that assumes data points are generated from a mixture of several Gaussian distributions.

<font color = '#FF6500' font size="4">**_Step 2: Choosing the Right Metrics_**</font>

_We will evaluate each clustering algorithm using common metrics to choose the best one:_

>- **1- Silhouette Score:** Measures how similar points are within a cluster compared to other clusters.
>- **2- Davies-Bouldin Index:** Measures the average similarity ratio of each cluster with the one that is most similar to it (lower is better).
>- **3- Inertia (KMeans only):** Measures the within-cluster sum of squares (the lower, the better for K-Means).

In [ ]:
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Function to evaluate and return the best clustering model and all models with their metrics
def evaluate_clustering_models(rfm_scaled):
    models = {}
    
    # K-Means
    best_score = -float('inf')  # Combined evaluation metric
    for k in range(2, 10):
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(rfm_scaled)
        silhouette_avg = silhouette_score(rfm_scaled, labels)
        davies_bouldin = davies_bouldin_score(rfm_scaled, labels)
        combined_score = silhouette_avg - davies_bouldin
        
        print(f'[KMeans] k={k}, Silhouette Score: {silhouette_avg}, Davies-Bouldin Score: {davies_bouldin}, Combined Score: {combined_score}')
        
        if combined_score > best_score:
            best_score = combined_score
            models['KMeans'] = {
                'model': kmeans,
                'combined_score': combined_score,
                'silhouette_avg': silhouette_avg,
                'davies_bouldin': davies_bouldin
            }

    # DBSCAN
    best_score = -float('inf')
    for eps in [0.5, 1.0, 1.5]:
        for min_samples in [5, 10, 20]:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(rfm_scaled)
            if len(set(labels)) > 1:  # Only calculate silhouette score if more than one cluster is found
                silhouette_avg = silhouette_score(rfm_scaled, labels)
                davies_bouldin = davies_bouldin_score(rfm_scaled, labels)
                combined_score = silhouette_avg - davies_bouldin
                
                print(f'[DBSCAN] eps={eps}, min_samples={min_samples}, Silhouette Score: {silhouette_avg}, Davies-Bouldin Score: {davies_bouldin}, Combined Score: {combined_score}')
                
                if combined_score > best_score:
                    best_score = combined_score
                    models['DBSCAN'] = {
                        'model': dbscan,
                        'combined_score': combined_score,
                        'silhouette_avg': silhouette_avg,
                        'davies_bouldin': davies_bouldin
                    }

    # Agglomerative Clustering
    best_score = -float('inf')
    for n_clusters in range(2, 10):
        for linkage in ['ward', 'complete', 'average']:
            agglomerative = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage)
            labels = agglomerative.fit_predict(rfm_scaled)
            silhouette_avg = silhouette_score(rfm_scaled, labels)
            davies_bouldin = davies_bouldin_score(rfm_scaled, labels)
            combined_score = silhouette_avg - davies_bouldin
            
            print(f'[Agglomerative] n_clusters={n_clusters}, linkage={linkage}, Silhouette Score: {silhouette_avg}, Davies-Bouldin Score: {davies_bouldin}, Combined Score: {combined_score}')
            
            if combined_score > best_score:
                best_score = combined_score
                models['Agglomerative'] = {
                    'model': agglomerative,
                    'combined_score': combined_score,
                    'silhouette_avg': silhouette_avg,
                    'davies_bouldin': davies_bouldin
                }

    # Gaussian Mixture Model (GMM)
    best_score = -float('inf')
    for n_components in range(2, 10):
        gmm = GaussianMixture(n_components=n_components, random_state=42)
        labels = gmm.fit_predict(rfm_scaled)
        silhouette_avg = silhouette_score(rfm_scaled, labels)
        davies_bouldin = davies_bouldin_score(rfm_scaled, labels)
        combined_score = silhouette_avg - davies_bouldin
        
        print(f'[GMM] n_components={n_components}, Silhouette Score: {silhouette_avg}, Davies-Bouldin Score: {davies_bouldin}, Combined Score: {combined_score}')
        
        if combined_score > best_score:
            best_score = combined_score
            models['GMM'] = {
                'model': gmm,
                'combined_score': combined_score,
                'silhouette_avg': silhouette_avg,
                'davies_bouldin': davies_bouldin
            }

    # Select the best model based on the highest combined score
    best_model_name = max(models, key=lambda key: models[key]['combined_score'])
    best_model = models[best_model_name]
    
    print(f'\nBest Model: {best_model_name}, Silhouette Avg: {best_model["silhouette_avg"]}, Davies Bouldin: {best_model["davies_bouldin"]}, Combined Score: {best_model["combined_score"]}')
    
    return best_model, models

# Run the evaluation and get the best model along with all models
best_model, models = evaluate_clustering_models(rfm_scaled)

# Assign cluster labels to the dataframe
rfm_df['KMeans_Cluster'] = models['KMeans']['model'].labels_
rfm_df['DBSCAN_Cluster'] = models['DBSCAN']['model'].labels_
rfm_df['Hierarchical_Cluster'] = models['Agglomerative']['model'].labels_
rfm_df['GMM_Cluster'] = models['GMM']['model'].predict(rfm_scaled)

# Saving the resulting dataframe to a CSV file
rfm_df.to_csv("RFM_Clustering_Results.csv", index=False)
print("The Dataset RFM_Clustering_Results.csv has been created successfully")

In [ ]:
rfm_df.head()

In [ ]:
# Cluster labeling map for all algorithms
cluster_labels = {
    -1: 'Noise',
    0: 'Hibernating',
    1: 'At Risk',
    2: 'Champions',
    3: 'Loyal',
    4: 'Best',
    5: 'Greedy'
}

# Function to map cluster numbers to labels
def label_clusters(cluster_column, algorithm_name):
    return cluster_column.map(cluster_labels)

# Step to evaluate clustering models (assuming you have the evaluate_clustering_models() function already implemented)
# best_model, models = evaluate_clustering_models(rfm_scaled)

# Adding cluster labels for each algorithm
rfm_df['KMeans_Cluster_Label'] = label_clusters(rfm_df['KMeans_Cluster'], 'KMeans')
rfm_df['DBSCAN_Cluster_Label'] = label_clusters(rfm_df['DBSCAN_Cluster'], 'DBSCAN')
rfm_df['Hierarchical_Cluster_Label'] = label_clusters(rfm_df['Hierarchical_Cluster'], 'Hierarchical')
rfm_df['GMM_Cluster_Label'] = label_clusters(rfm_df['GMM_Cluster'], 'GMM')

# Selecting and saving the final dataframe with labels
columns_to_save = [
    'CustomerID', 'Recency', 'Frequency', 'Monetary', 'RFM_Cluster', 'Label',
    'KMeans_Cluster', 'KMeans_Cluster_Label',
    'DBSCAN_Cluster', 'DBSCAN_Cluster_Label',
    'Hierarchical_Cluster', 'Hierarchical_Cluster_Label',
    'GMM_Cluster', 'GMM_Cluster_Label'
]

# Saving the result to a CSV file
rfm_df[columns_to_save].to_csv("RFM_Clustering_Labeled_Results.csv", index=False)
print("The Dataset with Cluster Labels has been created successfully.")

In [ ]:
rfm_df.head()

In [ ]:
# Selecting the specified columns to display in the DataFrame
columns_to_display = [
    'CustomerID', 'Recency', 'Frequency', 'Monetary', 
    'RFM_Cluster', 'Label', 'Hierarchical_Cluster', 'Hierarchical_Cluster_Label'
]

# Display the DataFrame with the selected columns
rfm_display_df = rfm_df[columns_to_display]
rfm_display_df

In [ ]:
# Merging transaction data with the RFM clustering results
merged_df = pd.merge(transactions_df, rfm_df, on='CustomerID', how='left')

# Merge merchant categories as well to contain transaction data, RFM segmentation, and merchant categories
merged_df = pd.merge(merged_df, categories_df, left_on='MerchantName', right_on='MerchantName', how='left')

merged_df

In [ ]:
# Selecting the specified columns to display in the DataFrame
columns_to_display = [
    'CustomerID', 'MerchantName', 'Category', 'TransactionRank', 'TransactionRedeemedPoints', 'TransactionValue', 'TransactionFrom(days)',
    'CustomerLastTransactionFrom(days)', 'Recency', 'Frequency', 'Monetary', 
    'RFM_Cluster', 'Label','Hierarchical_Cluster', 'Hierarchical_Cluster_Label'
]

# Display the DataFrame with the selected columns
merged_labeled_df = merged_df[columns_to_display]
merged_labeled_df

# Saving the result to a CSV file
merged_labeled_df.to_csv("RFM_Clustering_Labeled_Final.csv", index=False)
print("The Dataset 'RFM_Clustering_Labeled_Final.csv' has been created successfully.")

In [ ]:
# Aggregate transaction data by CustomerID and MerchantName
# This calculates the total amount spent and the count of transactions for each customer-merchant pair
merchant_data = merged_df.groupby(['CustomerID', 'MerchantName']).agg({
    'TransactionValue': 'sum',  # Total value spent
    'TransactionRank': 'count'  # Transaction frequency
}).reset_index()

# Rename columns for clarity
merchant_data.columns = ['CustomerID', 'MerchantName', 'Total_Spent', 'Transaction_Count']

# Group by 'Merchant' and sum their 'TransactionValue', then sort the results
top_10_unique_merchants = merchant_data.groupby('MerchantName')['Total_Spent'].sum().reset_index()

# Sort by 'TransactionValue' and display the top 10 merchants
top_10_unique_merchants = top_10_unique_merchants.sort_values(by='Total_Spent', ascending=False).head(10)

top_10_unique_merchants

In [ ]:
# Calculate total value spent and transaction frequency
customer_purchase_history = merchant_data.groupby('CustomerID').agg({
    'Total_Spent': 'sum',
    'Transaction_Count': 'sum'
}).reset_index()

# Display the summary of customer purchase history
print(customer_purchase_history.head())

In [ ]:
# Selecting the specified columns to display in the DataFrame
columns_to_display = [
    'CustomerID', 'MerchantName', 'Category', 'TransactionRank', 'TransactionRedeemedPoints', 'TransactionValue', 'TransactionFrom(days)',
    'CustomerLastTransactionFrom(days)', 'Recency', 'Frequency', 'Monetary'
]

# Display the DataFrame with the selected columns
RFM_deploy_df = merged_df[columns_to_display]
RFM_deploy_df

# Saving the result to a CSV file
RFM_deploy_df.to_csv("RFM_Deployment.csv", index=False)
print("The Dataset 'RFM_Deployment.csv' has been created successfully.")

In [ ]:
%%writefile RFM.py
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler

# Function to map cluster numbers to meaningful labels
def assign_cluster_labels(cluster_numbers):
    label_mapping = {
        0: 'Hibernating',
        1: 'At Risk',
        2: 'Champions',
        3: 'Loyal',
        4: 'Best',
        5: 'Greedy'
    }
    return [label_mapping.get(cluster, 'Unknown') for cluster in cluster_numbers]

# Title of the app
st.title('Customer Segmentation Using RFM Analysis')

# Sidebar for file upload
st.sidebar.header('Upload CSV File')
st.sidebar.markdown("Upload a CSV file containing customer transaction data. Ensure it has columns for Recency, Frequency, and Monetary values.")

# File uploader widget
uploaded_file = st.sidebar.file_uploader("Upload a CSV file", type="csv")

if uploaded_file is not None:
    try:
        # Load the uploaded CSV file
        df = pd.read_csv(uploaded_file)
        if df.empty:
            st.error("Uploaded file is empty. Please upload a valid CSV file.")
        else:
            st.write("Dataset Preview:")
            st.dataframe(df.head())

            # Select columns for RFM analysis
            st.sidebar.header('Select Columns for RFM Analysis')
            recency_col = st.sidebar.selectbox('Select Recency Column', df.columns)
            frequency_col = st.sidebar.selectbox('Select Frequency Column', df.columns)
            monetary_col = st.sidebar.selectbox('Select Monetary Column', df.columns)

            # Check if selected columns exist in the dataset
            if recency_col not in df.columns or frequency_col not in df.columns or monetary_col not in df.columns:
                st.error("Selected columns are not in the dataset. Please check your selection.")
            else:
                # Allow the user to select the number of clusters
                num_clusters = st.sidebar.slider("Select the number of clusters", min_value=2, max_value=10, value=4)

                # Scaling the RFM values
                scaler = StandardScaler()
                df_rfm = df[[recency_col, frequency_col, monetary_col]]
                df_rfm_scaled = scaler.fit_transform(df_rfm)

                # Allow user to select the clustering algorithm
                st.sidebar.header('Select Clustering Algorithm')
                cluster_algo = st.sidebar.selectbox('Select Algorithm', ('KMeans', 'Hierarchical Clustering'))

                # Dynamic file name based on selected algorithm
                if cluster_algo == 'KMeans':
                    # Apply KMeans clustering
                    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
                    df['Cluster'] = kmeans.fit_predict(df_rfm_scaled)  # KMeans Clusters
                    df['Cluster_Label'] = assign_cluster_labels(df['Cluster'])  # Assign meaningful labels
                    file_name = "segmented_data_KMeans.csv"  # Set file name based on algorithm
                    # Define columns for the CSV file in the KMeans case
                    columns_to_include = [
                        'CustomerID', 'MerchantName', 'Category', 'TransactionRank', 'TransactionRedeemedPoints', 
                        'TransactionValue', 'TransactionFrom(days)', 'CustomerLastTransactionFrom(days)',
                        'Recency', 'Frequency', 'Monetary', 'Cluster', 'Cluster_Label'
                    ]
                elif cluster_algo == 'Hierarchical Clustering':
                    # Apply Hierarchical Clustering
                    hierarchical = AgglomerativeClustering(n_clusters=num_clusters)
                    df['Cluster'] = hierarchical.fit_predict(df_rfm_scaled)  # Hierarchical Clusters
                    df['Cluster_Label'] = assign_cluster_labels(df['Cluster'])  # Assign meaningful labels
                    file_name = "segmented_data_Hierarchical.csv"  # Set file name based on algorithm
                    # Define columns for the CSV file in the Hierarchical Clustering case
                    columns_to_include = [
                        'CustomerID', 'MerchantName', 'Category', 'TransactionRank', 'TransactionRedeemedPoints', 
                        'TransactionValue', 'TransactionFrom(days)', 'CustomerLastTransactionFrom(days)',
                        'Recency', 'Frequency', 'Monetary', 'Cluster', 'Cluster_Label'
                    ]

                # Generate the CSV file for download
                segmented_df = df[columns_to_include]
                
                # Encode to CSV
                segmented_csv = segmented_df.to_csv(index=False).encode('utf-8')

                # Display the segmented data
                st.write("Segmented Data Preview:")
                st.dataframe(segmented_df)  # Now showing the selected columns only

                # Visualization of clusters
                st.header('Cluster Visualization')
                fig, ax = plt.subplots()
                scatter = ax.scatter(df[recency_col], df[frequency_col], c=df['Cluster'], cmap='viridis')
                legend1 = ax.legend(*scatter.legend_elements(), title="Clusters")
                ax.add_artist(legend1)
                plt.xlabel('Recency')
                plt.ylabel('Frequency')
                st.pyplot(fig)

                # Additional visualizations using seaborn (Cluster Distributions)
                st.header('Cluster Value Distributions')
                for cluster in df['Cluster'].unique():
                    st.write(f"Cluster {cluster} Distributions:")
                    # Create a figure and axes
                    fig, ax = plt.subplots()
                    sns.boxplot(data=df[df['Cluster'] == cluster][[recency_col, frequency_col, monetary_col]], ax=ax)
                    # Pass the figure to st.pyplot
                    st.pyplot(fig)

                # Allow user to download segmented data with dynamic file name
                st.sidebar.header('Download Segmented Data')
                st.sidebar.download_button(label="Download CSV", data=segmented_csv, file_name=file_name, mime='text/csv')

    except Exception as e:
        st.error(f"Error loading the file: {e}")
else:
    st.write("Please upload a CSV file to proceed.")